In [ ]:
import shutil
from pathlib import Path

import matplotlib.pyplot as plt
import tifffile
from careamics_portfolio import PortfolioManager

from careamics import CAREamist


### Import Dataset Portfolio

In [ ]:
# Explore portfolio
portfolio = PortfolioManager()
print(portfolio.denoising)

In [ ]:
# Download files
root_path = Path("./data")
files = portfolio.denoising.N2V_SEM.download(root_path)
print(f"List of downloaded files: {files}")

### Visualize training data

In [ ]:
# Load images
train_image = tifffile.imread(files[0])
print(f"Train image shape: {train_image.shape}")
plt.imshow(train_image, cmap="gray")

### Visualize validation data

In [ ]:
val_image = tifffile.imread(files[1])
print(f"Validation image shape: {val_image.shape}")
plt.imshow(val_image, cmap="gray")

In [ ]:
# Set paths

data_path = Path(root_path / "n2v_sem")
train_path = data_path / "train"
val_path = data_path / "val"

train_path.mkdir(parents=True, exist_ok=True)
val_path.mkdir(parents=True, exist_ok=True)

shutil.copy(root_path / files[0], train_path / "train_image.tif")
shutil.copy(root_path / files[1], val_path / "val_image.tif")

#### Initialize the Model

Create a Pytorch Lightning module

Please take as look at the [documentation](https://careamics.github.io) to see the full list of parameters and configuration options

In [ ]:
engine = CAREamist(source="n2n_2D_SEM.yml")

### Part 3. Run training 

We need to specify the paths to training and validation data

In [ ]:
engine.train(train_source=train_path, val_source=val_path,
             train_target=train_path, val_target=val_path,)

### Run prediction


In [ ]:
preds = engine.predict(source=train_path, tile_size=(256, 256))

### Visualize the prediction

In [ ]:
plt.imshow(preds[0].squeeze(), cmap="gray")